In [63]:
import pmdarima as pm
import pandas as pd
from random import sample, choice
import matplotlib.pyplot as plt
import numpy as np

In [38]:
dataset = pd.DataFrame(pm.datasets.load_gasoline(), columns=['gasoline'])

In [39]:
dataset['exog1'] = dataset.shift(-2)/100

In [40]:
dataset['exog1'][-2:] = [93.05, 93.88]

In [66]:
dataset['exog2'] = abs(np.random.randn(dataset.shape[0]))

In [67]:
dataset.head()

,gasoline,exog1,exog2
0,6621.0,65.82,0.636681
1,6433.0,72.24,0.232264
2,6582.0,68.75,0.009227
3,7224.0,69.47,0.648533
4,6875.0,73.28,0.464540


In [68]:
dataset.corr()

,gasoline,exog1,exog2
gasoline,1.000000,0.838285,-0.003887
exog1,0.838285,1.000000,-0.003150
exog2,-0.003887,-0.003150,1.000000


In [69]:
dataset.shape

(745, 3)